## Extensões na Prática

- O que são extensões?
- Documentação [https://duckdb.org/docs/extensions/overview.html](https://duckdb.org/docs/extensions/overview.html)
- Oficiais [https://duckdb.org/docs/extensions/official_extensions](https://duckdb.org/docs/extensions/official_extensions)
- Hands On

In [1]:
#arrow

In [ ]:
#autocomplete	

In [ ]:
#aws

In [ ]:
#httpfs

In [ ]:
#azure

In [ ]:
#delta

In [ ]:
#excel

In [ ]:
#fts

In [ ]:
#iceberg

In [ ]:
#postgres

In [ ]:
#mysql